In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import re
from HelperFunctions import *

In [ ]:
def getData(directory):
    end_time = 24*3600 # 24 hours
    for sdir_path, sdir in [(sdir.path, sdir.name) for sdir in os.scandir(directory) if sdir.is_dir()]:
        for root, subdirs, files in os.walk(sdir_path):
            for file in files:
                if file == "output_trips.csv.gz":
                    df_trips = pd.read_csv(
                        os.path.join(root, file), sep=";", compression="gzip"
                    ).loc[:, ["person", "dep_time", "trav_time", "wait_time", "traveled_distance", "modes"]]
                    df_trips["dep_time"] = df_trips["dep_time"].apply(timestmp2sec)
                    df_trips = df_trips[df_trips['dep_time'] < end_time]
                    df_trips["trav_time"] = df_trips["trav_time"].apply(timestmp2sec)
                    df_trips["wait_time"] = df_trips["wait_time"].apply(timestmp2sec)
                    df_trips.set_index("person", inplace=True)
                if file == "0.drt_trips_drt.csv":
                    df_drt = pd.read_csv(os.path.join(root, file), sep=";").loc[
                        :, ["departureTime", "personId", "waitTime", "travelDistance_m", "direcTravelDistance_m"]
                    ]
                    df_drt = df_drt[df_drt['departureTime'] < end_time]
                    df_drt = df_drt.groupby("personId").sum()
                if file == "0.drt_occupancy_time_profiles_drt.txt":
                    df_occ = pd.read_csv(os.path.join(root, file), sep="\t")
                    df_occ['time'] = df_occ['time'].apply(timestmphm2sec)
                    df_occ = df_occ[df_occ['time'] < end_time]

        yield int(sdir), df_trips, df_drt, df_occ

In [ ]:
def getAverageOcc(df):
    weights = np.zeros(len(df.columns))
    weights[1:] = np.arange(0,len(df.columns)-1)
    
    pass_sum = df.sum(axis=1)
    return (df.dot(weights)/pass_sum).mean()

In [ ]:
path = "/home/helge/Programs/matsim/matsim_results/OwnIntermodalAccEgr/OptDrtCount/bim_gamma1"

count = 0
vehicle_list = []
av_occupancies = []
av_detours = []
total_dists = []
for vehicles, df_trips, df_drt_trips, df_occ in getData(path):
    print('Number of vehicles: ', vehicles)
    df_trips = df_trips.groupby("person").agg(
        {
            "dep_time": lambda x: x.iloc[0],
            "trav_time": np.sum,
            "wait_time": np.sum,
            "traveled_distance": np.sum,
            "modes": combineModesSeriesStr,
        }
    )
    vehicle_list.append(vehicles)
    av_occupancies.append(getAverageOcc(df_occ.drop(columns = 'time')))
    av_detours.append(
        (
            df_drt_trips["travelDistance_m"] / df_drt_trips["direcTravelDistance_m"]
        ).mean()
    )
    total_dists.append(df_trips['traveled_distance'].sum())
        
vehicle_list = np.array(vehicle_list)
av_occupancies = np.array(av_occupancies)
av_detours = np.array(av_detours)
total_dists = np.array(total_dists)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1)

ax1.scatter(vehicle_list, av_occupancies/av_detours)
ax1.set_ylabel(r'$\langle b\rangle / \langle\delta\rangle$')
ax1.set_xlabel('Number of drt vehicles')
ax2.scatter(vehicle_list, total_dists)
ax2.set_ylabel('total driven distance')
ax2.set_xlabel('Number of drt vehicles')
fig.tight_layout()

plt.show()